# Import Libraries

In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import scipy

In [2]:
path = r'/Users/bazillaimran/Documents/02-2023 Instacart Basket Analysis/02 Data'

In [3]:
df_customers = pd.read_csv(os.path.join(path,'Original Data','customers.csv'))

# Data Wrangling: rename columns with logical names and drop unwanted columns 

In [4]:
df_customers.head()

,user_id,First Name,Surnam,Gender,STATE,Age,date_joined,n_dependants,fam_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374


In [5]:
df_customers.shape

(206209, 10)

In [6]:
# rename columns
df_customers.rename(columns = {'Surnam':'Last Name','fam_status':'family_status','STATE':'State','n_dependants':'dependants'}, inplace = True)

In [7]:
df_customers.head()

,user_id,First Name,Last Name,Gender,State,Age,date_joined,dependants,family_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374


All columns contain valuable information, there is no necessity to eliminate any of them.

# Data quality and consistency checks: missing values, duplicates, and mixed-data conversion

In [8]:
df_customers.describe()

,user_id,Age,dependants,income
count,206209.000000,206209.000000,206209.000000,206209.000000
mean,103105.000000,49.501646,1.499823,94632.852548
std,59527.555167,18.480962,1.118433,42473.786988
min,1.000000,18.000000,0.000000,25903.000000
25%,51553.000000,33.000000,0.000000,59874.000000
50%,103105.000000,49.000000,1.000000,93547.000000
75%,154657.000000,66.000000,3.000000,124244.000000
max,206209.000000,81.000000,3.000000,593901.000000


Income column: If the mean and median are close in value, it suggests that the income distribution is relatively symmetric. In other words, the data is evenly distributed around the center.

In [9]:
# Address mixed-data types

In [10]:
df_customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 206209 entries, 0 to 206208
Data columns (total 10 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   user_id        206209 non-null  int64 
 1   First Name     194950 non-null  object
 2   Last Name      206209 non-null  object
 3   Gender         206209 non-null  object
 4   State          206209 non-null  object
 5   Age            206209 non-null  int64 
 6   date_joined    206209 non-null  object
 7   dependants     206209 non-null  int64 
 8   family_status  206209 non-null  object
 9   income         206209 non-null  int64 
dtypes: int64(4), object(6)
memory usage: 15.7+ MB


No mixed datatype in each column. However, a more appropriate data type for the date_joined column would be datetime data type.

In [11]:
# Change data type for object to datetime for date_joined column
df_customers['date_joined'] = pd.to_datetime(df_customers['date_joined'])

In [12]:
df_customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 206209 entries, 0 to 206208
Data columns (total 10 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   user_id        206209 non-null  int64         
 1   First Name     194950 non-null  object        
 2   Last Name      206209 non-null  object        
 3   Gender         206209 non-null  object        
 4   State          206209 non-null  object        
 5   Age            206209 non-null  int64         
 6   date_joined    206209 non-null  datetime64[ns]
 7   dependants     206209 non-null  int64         
 8   family_status  206209 non-null  object        
 9   income         206209 non-null  int64         
dtypes: datetime64[ns](1), int64(4), object(5)
memory usage: 15.7+ MB


In [13]:
# Check for duplicates
# extracting duplicate from df_customers and store it in dataframe df_dups
df_dups = df_customers[df_customers.duplicated()]

In [14]:
df_dups

,user_id,First Name,Last Name,Gender,State,Age,date_joined,dependants,family_status,income


No duplicates found.

In [15]:
# Check for missing values
df_customers.isnull().sum()

user_id              0
First Name       11259
Last Name            0
Gender               0
State                0
Age                  0
date_joined          0
dependants           0
family_status        0
income               0
dtype: int64

The output shows there are 11259 missing values in the First Name column.

In [16]:
# Viewing missing values: create a subset of the dataframe containing only the missing values
df_nan=df_customers[df_customers['First Name'].isnull()==True]

In [17]:
df_nan

,user_id,First Name,Last Name,Gender,State,Age,date_joined,dependants,family_status,income
53,76659,NaN,Gilbert,Male,Colorado,26,2017-01-01,2,married,41709
73,13738,NaN,Frost,Female,Louisiana,39,2017-01-01,0,single,82518
82,89996,NaN,Dawson,Female,Oregon,52,2017-01-01,3,married,117099
99,96166,NaN,Oconnor,Male,Oklahoma,51,2017-01-01,1,married,155673
105,29778,NaN,Dawson,Female,Utah,63,2017-01-01,3,married,151819
...,...,...,...,...,...,...,...,...,...,...
206038,121317,NaN,Melton,Male,Pennsylvania,28,2020-03-31,3,married,87783
206044,200799,NaN,Copeland,Female,Hawaii,52,2020-04-01,2,married,108488
206090,167394,NaN,Frost,Female,Hawaii,61,2020-04-01,1,married,45275
206162,187532,NaN,Floyd,Female,California,39,2020-04-01,0,single,56325


Since the missing information is only in the "First Name" column for 11,259 rows, it's better to keep them. Deleting these rows could make our data less reliable, so it's a good idea to leave them as they are. This way, we keep more data and make sure our analysis is based on as much information as possible. However, we can enhance the comprehensibility of our data by replacing these missing values with 'Unknown'.

In [18]:
# replace NaN with Unknown
df_customers['First Name'].fillna('Unknown', inplace = True)

In [19]:
df_customers.isnull().sum()

user_id          0
First Name       0
Last Name        0
Gender           0
State            0
Age              0
date_joined      0
dependants       0
family_status    0
income           0
dtype: int64

In [20]:
df_customers['First Name'].value_counts()

First Name
Unknown    11259
Marilyn     2213
Barbara     2154
Todd        2113
Jeremy      2104
           ...  
Merry        197
Eugene       197
Garry        191
Ned          186
David        186
Name: count, Length: 208, dtype: int64

# Combining customer data with prepared data


In [21]:
# Load prepared data
df_all=pd.read_pickle(os.path.join(path, 'Prepared Data', 'ords_prods_grouped.pkl'))

In [22]:
# Combine df_cutomer with df_all
# Check shape of df_customer
df_customers.shape

(206209, 10)

In [23]:
# Check shape of df_all
df_all.shape

(32404859, 24)

In [24]:
# check data type and determine a suitable way to combine the 2 dataframes
df_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32404859 entries, 0 to 32404858
Data columns (total 24 columns):
 #   Column                  Dtype   
---  ------                  -----   
 0   product_id              int64   
 1   product_name            object  
 2   aisle_id                int64   
 3   department_id           int64   
 4   prices                  float64 
 5   order_id                int64   
 6   user_id                 int64   
 7   order_number            int64   
 8   orders_day_of_week      int64   
 9   order_hour_of_day       int64   
 10  days_since_prior_order  float64 
 11  add_to_cart_order       int64   
 12  reordered               int64   
 13  _merge                  category
 14  price_range_loc         object  
 15  busiest_day             object  
 16  busiest_days            object  
 17  busiest_period_of_day   object  
 18  max_order               int64   
 19  loyalty_flag            object  
 20  average_spend           float64 
 21  spende

In [25]:
df_customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 206209 entries, 0 to 206208
Data columns (total 10 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   user_id        206209 non-null  int64         
 1   First Name     206209 non-null  object        
 2   Last Name      206209 non-null  object        
 3   Gender         206209 non-null  object        
 4   State          206209 non-null  object        
 5   Age            206209 non-null  int64         
 6   date_joined    206209 non-null  datetime64[ns]
 7   dependants     206209 non-null  int64         
 8   family_status  206209 non-null  object        
 9   income         206209 non-null  int64         
dtypes: datetime64[ns](1), int64(4), object(5)
memory usage: 15.7+ MB


In [26]:
# Drop _merge columns in df_all (not necessary) 
df_all = df_all.drop(columns = ['_merge'])

Merge the to dataframes by using the user_id columns (same data type of int64). 

In [27]:
# Merge by column user_id since it's present in both dataframes 
ords_prods_customers_merged = df_all.merge(df_customers, on = 'user_id', indicator = True)

In [28]:
# Confirm the results of the merge using the merge flag.
ords_prods_customers_merged.head()

,product_id,product_name,aisle_id,department_id,prices,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,...,First Name,Last Name,Gender,State,Age,date_joined,dependants,family_status,income,_merge
0,1,Chocolate Sandwich Cookies,61,19,5.8,3139998,138,28,6,11,...,Charles,Cox,Male,Minnesota,81,2019-08-01,1,married,49620,both
1,1,Chocolate Sandwich Cookies,61,19,5.8,1977647,138,30,6,17,...,Charles,Cox,Male,Minnesota,81,2019-08-01,1,married,49620,both
2,907,Premium Sliced Bacon,106,12,20.0,3160996,138,1,5,13,...,Charles,Cox,Male,Minnesota,81,2019-08-01,1,married,49620,both
3,907,Premium Sliced Bacon,106,12,20.0,2254091,138,10,5,14,...,Charles,Cox,Male,Minnesota,81,2019-08-01,1,married,49620,both
4,1000,Apricots,18,10,12.9,505689,138,9,6,12,...,Charles,Cox,Male,Minnesota,81,2019-08-01,1,married,49620,both


In [29]:
#Checking the values in the _merge column using merge flag
ords_prods_customers_merged['_merge'].value_counts()

_merge
both          32404859
left_only            0
right_only           0
Name: count, dtype: int64

In [30]:
ords_prods_customers_merged.shape

(32404859, 33)

# Exporting New Dataframe as a pickle file

In [31]:
ords_prods_customers_merged = ords_prods_customers_merged.to_pickle(os.path.join(path,'Prepared Data', 'ords_prods_customers_merged.pkl'))